In [98]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import math

import statsmodels.formula.api as smf
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, backend

pd.options.display.float_format = "{:,.2f}".format

In [84]:
data_dir = '../data'
df = pd.read_csv(os.path.join(data_dir, 'all_merged_12_06.csv'))
df = df.drop(['Unnamed: 0'], axis=1)

#Eliminate areas that are not enough burnable
df = df[df['burnable_frac']>=0.9]

#Reorganize
df = df.drop(columns=['burnable_frac'])
df = df.reindex(columns=['lon','lat','year', 'month', 'burned_area', 'temperature', 'precipitation', 'dry_days', 'land_cover', 'country'])
df = df.dropna()
df = df.reset_index()
df = df.drop(['index'], axis=1)
df = df.sort_values(['lon','lat','year','month'])
df['location'] = ' (' + df['lon'].map(str) + ',' + df['lat'].map(str) + ')'
df = df.drop(['lon','lat','year','month','land_cover','country',], axis=1)
df = df[df.groupby('location').location.transform('count')>=192]
df

,burned_area,temperature,precipitation,dry_days,location
1823855,0,4.90,106.70,18.73,"(-9.75,52.0)"
1823856,0,5.50,131.30,18.73,"(-9.75,52.0)"
1823857,0,5.90,130.40,18.73,"(-9.75,52.0)"
1823858,0,7.70,118.50,18.73,"(-9.75,52.0)"
1823859,0,11.40,47.50,18.73,"(-9.75,52.0)"
...,...,...,...,...,...
1823850,1,25.20,17.00,31.73,"(50.0,51.75)"
1823851,0,14.30,105.70,31.73,"(50.0,51.75)"
1823852,0,5.40,23.10,31.73,"(50.0,51.75)"
1823853,0,-3.40,49.90,31.73,"(50.0,51.75)"


In [105]:
df['burned_area'].value_counts()

0    1684069
1     149915
Name: burned_area, dtype: int64

In [85]:
seq_len = df['location'].value_counts()
seq_len

 (-9.75,52.0)     192
 (33.5,51.75)     192
 (33.75,36.25)    192
 (33.75,36.75)    192
 (33.75,37.0)     192
                 ... 
 (14.75,37.0)     192
 (14.75,37.25)    192
 (14.75,37.5)     192
 (14.75,37.75)    192
 (50.0,51.75)     192
Name: location, Length: 9552, dtype: int64

In [107]:
X_arr = df.drop(columns=['burned_area']).set_index('location').groupby('location').apply(pd.DataFrame.to_numpy).to_numpy()
X_arr = np.stack(X_arr)
y_arr = df[['burned_area','location']].set_index('location').groupby('location').apply(pd.DataFrame.to_numpy).to_numpy()
y_arr = np.stack(y_arr)
print(X_arr.shape)
print(y_arr.shape)

(9552, 192, 3)
(9552, 192, 1)


In [99]:
def recall(y_true, y_pred):
    y_true = backend.ones_like(y_true)
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    all_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))

    recall = true_positives / (all_positives + backend.epsilon())
    return recall

def precision(y_true, y_pred):
    y_true = backend.ones_like(y_true)
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))

    predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + backend.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+backend.epsilon()))

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

# 1- RNN Architecture
model = Sequential()
model.add(layers.LSTM(units=2, return_sequences=True, activation='tanh', input_shape=(192,3)))
model.add(layers.Dense(1, activation="sigmoid"))

# 2- Compilation
adam_opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

# 3- Fit
model.fit(X_arr, y_arr, epochs=100, verbose=1)

2023-06-13 01:08:13.962855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-13 01:08:13.965786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-13 01:08:13.970880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/100


2023-06-13 01:08:14.765666: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-13 01:08:14.772001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-13 01:08:14.774793: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4/4 [==============================] - 6s 96ms/step - loss: 0.3545 - accuracy: 0.9102
Epoch 2/100
4/4 [==============================] - 0s 87ms/step - loss: 0.3437 - accuracy: 0.9102
Epoch 3/100
4/4 [==============================] - 0s 86ms/step - loss: 0.3338 - accuracy: 0.9102
Epoch 4/100
4/4 [==============================] - 0s 89ms/step - loss: 0.3248 - accuracy: 0.9102
Epoch 5/100
4/4 [==============================] - 0s 79ms/step - loss: 0.3156 - accuracy: 0.9102
Epoch 6/100
4/4 [==============================] - 0s 78ms/step - loss: 0.3086 - accuracy: 0.9102
Epoch 7/100
4/4 [==============================] - 0s 97ms/step - loss: 0.3027 - accuracy: 0.9102
Epoch 8/100
4/4 [==============================] - 0s 79ms/step - loss: 0.2986 - accuracy: 0.9102
Epoch 9/100
4/4 [==============================] - 0s 83ms/step - loss: 0.2958 - accuracy: 0.9102
Epoch 10/100
4/4 [==============================] - 0s 80ms/step - loss: 0.2939 - accuracy: 0.9102
Epoch 11/100
4/4 [=============